## Environment

In [2]:
!pip install scikit-optimize
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from sklearn.model_selection import train_test_split

     |████████████████████████████████| 102kB 4.2MB/s 


In [3]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, space_eval

In [4]:
import pandas as pd
import numpy as np
import time

In [5]:
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
!pip install tensorflow-datasets==3.2.1
import tensorflow_datasets as tfds

     |████████████████████████████████| 3.4MB 11.9MB/s 
  Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1


In [ ]:
for ex_dict in ds_numpy:
  print(ex_dict['image'])
  #print(ex_dict['labels'].shape)

## DATA

In [7]:
trainset, testset = keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [8]:
X_train, X_val, y_train, y_val = train_test_split(trainset[0], trainset[1], test_size=0.2, random_state=12345)
X_train, X_val = X_train / 255.0, X_val / 255.0

In [9]:
X_train, X_train_early_stop, y_train, y_train_early_stop = train_test_split(X_train, y_train, test_size=0.05, random_state=1234)

## TUNNING

In [10]:
 params_grid = {'layers_': hp.choice('layers_', [2, 3, 4, 5, 6, 7, 8, 9, 10]),
                'learning_rate': hp.uniform('learning_rate', 0.00001, 0.1),
                'neurons_': hp.quniform('neurons_', 10, 500, 1),
                'dropout': hp.uniform('dropout', .25,.75)}

In [ ]:
def build_model(layers_=2, neurons_=10, learning_rate=0.001):

  model = keras.models.Sequential()
  model.add(layers.Flatten())

  for i in range(layers_):
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(neurons_, 
                           activation=keras.activations.elu, 
                           kernel_initializer=keras.initializers.he_normal, 
                           use_bias=False))

  model.add(layers.Dense(10, 
                         activation=keras.activations.softmax, 
                         kernel_initializer=keras.initializers.he_normal))

  model.compile(optimizer=keras.optimizers.SGD(lr=learning_rate, momentum=0.9),
               loss=keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

  return model

In [ ]:
keras_wrapper = keras.wrappers.scikit_learn.KerasClassifier(build_model, 
                                                            epochs=100,
                                                            validation_split=0.5
                                                            #validation_data=(X_train_early_stop, y_train_early_stop),
                                                            #keras.callbacks.EarlyStopping(patience=5)
                                                            )

In [ ]:
opt = BayesSearchCV(
    keras_wrapper,
    {'layers_': Integer(1, 200),
     'learning_rate': (1e-6, 1e-2, 'log-uniform'),
     'neurons_': Integer(1, 8),
     'epochs': [10, 15, 20, 25, 30],
     'dropout': hp.uniform('dropout', .25,.75),
    },
    n_iter=32,
    cv=3
)

fit_params = dict(callbacks=[keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)])

opt.fit(X_train, y_train, **fit_params)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_test, y_test))

TypeError: ignored

## Metod 2 using hyperopt

In [11]:
 params_grid = {'layers_': hp.choice('layers_', [2, 3, 4, 5, 6, 7, 8, 9, 10]),
                'learning_rate': hp.uniform('learning_rate', 0.00001, 0.1),
                'neurons_': hp.quniform('neurons_', 10, 500, 1),
                'dropout': hp.uniform('dropout', 0,.75)}

In [12]:
def objective(params):
    
    print('\n', params)
    
    model = keras.models.Sequential()
    model.add(layers.Flatten())

    for i in range(params['layers_']):
      model.add(layers.BatchNormalization())
      model.add(layers.Dropout(rate=params['dropout']))
      model.add(layers.Dense(int(params['neurons_']), 
                            activation=keras.activations.elu, 
                            kernel_initializer=keras.initializers.he_normal, 
                            use_bias=False))

    model.add(layers.Dropout(rate=params['dropout']))
    model.add(layers.Dense(10, 
                          activation=keras.activations.softmax, 
                          kernel_initializer=keras.initializers.he_normal))

    model.compile(optimizer=keras.optimizers.SGD(lr=params['learning_rate'], momentum=0.9),
                loss=keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])
    
    model.fit(X_train,
              y_train,
              epochs=100,
              validation_data=(X_train_early_stop, y_train_early_stop),
              callbacks=keras.callbacks.EarlyStopping(patience=5),
              verbose=0)

    val_score = model.evaluate(X_val, y_val,
                              verbose=0)
    
    return {
        'loss':  val_score[0],
        'accuracy': val_score[1],
        'status': STATUS_OK,
        'eval_time': time.time()
        }

In [ ]:
# Benchmark
objective({'learning_rate':0.01, 'layers_': 3, 'neurons_': 20})


 {'learning_rate': 0.01, 'layers_': 3, 'neurons_': 20}


{'accuracy': 0.9602500200271606,
 'eval_time': 1610490715.472236,
 'loss': 0.14226077497005463,
 'status': 'ok'}

In [ ]:
trials = Trials()

best = fmin(fn=objective,
           space=params_grid,
           algo=tpe.suggest,
           max_evals=50,
           trials=trials,
           verbose=True,
           show_progressbar=True)

{'dropout': 0.6993786071256669, 'layers_': 9, 'learning_rate': 0.07182211369351536, 'neurons_': 241.0}
{'dropout': 0.7467445253480463, 'layers_': 7, 'learning_rate': 0.07588864150403016, 'neurons_': 15.0}
{'dropout': 0.6047989266235354, 'layers_': 5, 'learning_rate': 0.05052696295218311, 'neurons_': 465.0}
{'dropout': 0.5447370240043307, 'layers_': 6, 'learning_rate': 0.016569990567445684, 'neurons_': 330.0}
{'dropout': 0.551632370016417, 'layers_': 7, 'learning_rate': 0.017975063245596584, 'neurons_': 113.0}
{'dropout': 0.5116187589185748, 'layers_': 5, 'learning_rate': 0.0901145877929097, 'neurons_': 230.0}
{'dropout': 0.7018962570869532, 'layers_': 7, 'learning_rate': 0.015308609522616244, 'neurons_': 335.0}
 12%|█▏        | 6/50 [15:52<1:51:58, 152.69s/it, best loss: 0.08728916198015213]

In [ ]:
pickle.dump(trials, open("myfile.p", "wb"))